In [ ]:
import io
import socket
import struct
import time
import picamera

camera = picamera.PiCamera()
camera.resolution = (640, 480) 
camera.rotation = 270
camera.start_preview()
time.sleep(2) 
client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

In [ ]:
stream = io.BytesIO()
for _ in camera.capture_continuous(stream, 'jpeg'):
    img_len = stream.tell() 
    client_socket.sendto(struct.pack('<L', img_len), ('192.168.0.94', 8000)) 
    stream.flush()  
    stream.seek(0) 
    while img_len: 
        img_len -= 65507 
        client_socket.sendto(stream.read(65507), ('192.168.0.94', 8000)) # 65507 is the max length of UDP Packet
        if img_len < 0 : break
    stream.seek(0)
    stream.truncate()
    time.sleep(1)